In [ ]:

# BOOTSTRAP CELL — RUN THIS FIRST AFTER EVERY RUNTIME RESET


import os
import random
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix
)

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

MODEL_VERSION = "FL with HE"

BASE_DIR = "/content/drive/MyDrive/FYP_FL_IDS"

RAW_DATA_DIR = os.path.join(BASE_DIR, "data/raw")
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, "data/processed")

MODELS_DIR = os.path.join(BASE_DIR, "models")
FL_DIR = os.path.join(BASE_DIR, "fl")
RESULTS_DIR = os.path.join(BASE_DIR, "results")

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(FL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)
print("GPU:", torch.cuda.get_device_name(0) if DEVICE == "cuda" else "CPU")

CONFIG = {
    "SEED": SEED,
    "NUM_CLIENTS": 3,
    "BATCH_SIZE": 128,
    "LOCAL_EPOCHS": 1,
    "ROUNDS": 25,
    "LEARNING_RATE": 1e-3,
    "SEQUENCE_LENGTH": 10,
    "NUM_CLASSES": 1,
    "DEVICE": DEVICE,
}


POS_WEIGHT = torch.tensor([5.0], device=DEVICE)
CRITERION = nn.BCEWithLogitsLoss(pos_weight=POS_WEIGHT)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Bootstrap completed successfully.")
print("Ready for Phase 5 (Model Definition).")


Mounted at /content/drive
Device: cpu
GPU: CPU
Bootstrap completed successfully.
Ready for Phase 5 (Model Definition).


In [ ]:
CONFIG = {
    "SEED": 42,
    "NUM_CLIENTS": 3,
    "BATCH_SIZE": 64,
    "LOCAL_EPOCHS": 1,
    "ROUNDS": 12,
    "LEARNING_RATE": 1e-3,
    "SEQUENCE_LENGTH": 10,
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
}

CONFIG


{'SEED': 42,
 'NUM_CLIENTS': 3,
 'BATCH_SIZE': 64,
 'LOCAL_EPOCHS': 1,
 'ROUNDS': 12,
 'LEARNING_RATE': 0.001,
 'SEQUENCE_LENGTH': 10,
 'DEVICE': 'cuda'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

BASE_DIR = "/content/drive/MyDrive/FYP_FL_IDS"

folders = [
    "data/raw",
    "data/processed",
    "models",
    "fl",
    "utils",
    "results/logs",
    "results/plots",
    "notebooks"
]

for folder in folders:
    path = os.path.join(BASE_DIR, folder)
    os.makedirs(path, exist_ok=True)

print("Project directory structure created.")



Project directory structure created.


In [ ]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [ ]:
!pip install -q \
    numpy \
    pandas \
    scikit-learn \
    imbalanced-learn \
    matplotlib \
    seaborn \
    tqdm


In [ ]:
# Install TenSEAL for Homomorphic Encryption
!pip install -q tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 67.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import sklearn
import torch
import matplotlib
import seaborn

print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Scikit-learn:", sklearn.__version__)
print("Torch:", torch.__version__)
print("Matplotlib:", matplotlib.__version__)
print("Seaborn:", seaborn.__version__)


NumPy: 2.0.2
Pandas: 2.2.2
Scikit-learn: 1.6.1
Torch: 2.9.0+cu126
Matplotlib: 3.10.0
Seaborn: 0.13.2


In [ ]:
import random
import torch

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print("Random seeds fixed to:", SEED)


Random seeds fixed to: 42


In [ ]:
import os
raw_data_path = os.path.join(BASE_DIR, "data/raw")
files = os.listdir(raw_data_path)

print("Files found:")
for f in files:
    print(" -", f)


Files found:
 - Wednesday-workingHours.pcap_ISCX.csv
 - Tuesday-WorkingHours.pcap_ISCX.csv
 - Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
 - Friday-WorkingHours-Morning.pcap_ISCX.csv
 - Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
 - Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv


In [ ]:
dataframes = []

for file in files:
    file_path = os.path.join(RAW_DATA_DIR, file)
    print(f"Loading {file} ...")

    df = pd.read_csv(file_path, low_memory=False)
    df["source_file"] = file   # keep traceability

    dataframes.append(df)

print("\nAll files loaded successfully.")


Loading Wednesday-workingHours.pcap_ISCX.csv ...
Loading Tuesday-WorkingHours.pcap_ISCX.csv ...
Loading Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv ...
Loading Friday-WorkingHours-Morning.pcap_ISCX.csv ...
Loading Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv ...
Loading Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv ...

All files loaded successfully.


In [ ]:
df_all = pd.concat(dataframes, axis=0, ignore_index=True)

print("Combined dataset shape:", df_all.shape)

print("Columns in dataset:\n")
for col in df_all.columns:
    print(col)




Combined dataset shape: (2012223, 80)
Columns in dataset:

 Destination Port
 Flow Duration
 Total Fwd Packets
 Total Backward Packets
Total Length of Fwd Packets
 Total Length of Bwd Packets
 Fwd Packet Length Max
 Fwd Packet Length Min
 Fwd Packet Length Mean
 Fwd Packet Length Std
Bwd Packet Length Max
 Bwd Packet Length Min
 Bwd Packet Length Mean
 Bwd Packet Length Std
Flow Bytes/s
 Flow Packets/s
 Flow IAT Mean
 Flow IAT Std
 Flow IAT Max
 Flow IAT Min
Fwd IAT Total
 Fwd IAT Mean
 Fwd IAT Std
 Fwd IAT Max
 Fwd IAT Min
Bwd IAT Total
 Bwd IAT Mean
 Bwd IAT Std
 Bwd IAT Max
 Bwd IAT Min
Fwd PSH Flags
 Bwd PSH Flags
 Fwd URG Flags
 Bwd URG Flags
 Fwd Header Length
 Bwd Header Length
Fwd Packets/s
 Bwd Packets/s
 Min Packet Length
 Max Packet Length
 Packet Length Mean
 Packet Length Std
 Packet Length Variance
FIN Flag Count
 SYN Flag Count
 RST Flag Count
 PSH Flag Count
 ACK Flag Count
 URG Flag Count
 CWE Flag Count
 ECE Flag Count
 Down/Up Ratio
 Average Packet Size
 Avg Fwd Segm

In [ ]:
DROP_COLUMNS = [
    "Flow ID",
    "Source IP",
    "Destination IP",
    "Timestamp"
]

existing_drop_cols = [c for c in DROP_COLUMNS if c in df_all.columns]

df_all.drop(columns=existing_drop_cols, inplace=True)

print("Dropped columns:", existing_drop_cols)
print("Remaining shape:", df_all.shape)

df_all.replace([np.inf, -np.inf], np.nan, inplace=True)
nan_count = df_all.isna().sum().sum()
print("Total NaN values in dataset:", nan_count)

numeric_cols = df_all.select_dtypes(include=[np.number]).columns

df_all[numeric_cols] = df_all[numeric_cols].fillna(
    df_all[numeric_cols].median()
)

print("NaN values after imputation:", df_all.isna().sum().sum())


Dropped columns: []
Remaining shape: (2012223, 80)
Total NaN values in dataset: 4446
NaN values after imputation: 0


In [ ]:
df_all.columns = df_all.columns.str.strip()
print(df_all["Label"].value_counts())


Label
BENIGN                        1454613
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64


In [ ]:
print(df_all["Label"].unique())
df_all["Label"] = df_all["Label"].astype(str).str.strip().str.lower()
print(df_all["Label"].unique())

import re

def clean_label(label):
    label = label.lower()
    label = label.strip()
    label = re.sub(r'[^a-z0-9\s]', ' ', label)  # remove special chars
    label = re.sub(r'\s+', ' ', label)          # normalize spaces
    return label

df_all["Label"] = df_all["Label"].astype(str).apply(clean_label)
print(df_all["Label"].unique())
df_all["Label"] = df_all["Label"].apply(
    lambda x: 0 if x == "benign" else 1
)
print(df_all["Label"].value_counts())


['BENIGN' 'DoS slowloris' 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye'
 'Heartbleed' 'FTP-Patator' 'SSH-Patator' 'Web Attack � Brute Force'
 'Web Attack � XSS' 'Web Attack � Sql Injection' 'Bot' 'PortScan' 'DDoS']
['benign' 'dos slowloris' 'dos slowhttptest' 'dos hulk' 'dos goldeneye'
 'heartbleed' 'ftp-patator' 'ssh-patator' 'web attack � brute force'
 'web attack � xss' 'web attack � sql injection' 'bot' 'portscan' 'ddos']
['benign' 'dos slowloris' 'dos slowhttptest' 'dos hulk' 'dos goldeneye'
 'heartbleed' 'ftp patator' 'ssh patator' 'web attack brute force'
 'web attack xss' 'web attack sql injection' 'bot' 'portscan' 'ddos']
Label
0    1454613
1     557610
Name: count, dtype: int64


In [ ]:
clean_path = os.path.join(PROCESSED_DATA_DIR, "cic_ids2017_clean_phase1.csv")
df_all.to_csv(clean_path, index=False)

print("Phase 1 cleaned dataset saved at:", clean_path)


Phase 1 cleaned dataset saved at: /content/drive/MyDrive/FYP_FL_IDS/data/processed/cic_ids2017_clean_phase1.csv


In [ ]:
import os
import pandas as pd

clean_path = os.path.join(PROCESSED_DATA_DIR, "cic_ids2017_clean_phase1.csv")

df = pd.read_csv(clean_path)

print("Dataset loaded.")
print("Shape:", df.shape)

X = df.drop(columns=["Label"])
y = df["Label"]

print("Feature matrix shape:", X.shape)
print("Label vector shape:", y.shape)
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Categorical columns:", categorical_cols)
if "source_file" in X.columns:
    X = X.drop(columns=["source_file"])
    print("Dropped column: source_file")

print("Updated feature shape:", X.shape)
X = X.astype("float32")

print("Converted features to float32.")

categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()

print("Categorical columns:", categorical_cols)


Dataset loaded.
Shape: (2012223, 80)
Feature matrix shape: (2012223, 79)
Label vector shape: (2012223,)
Categorical columns: ['source_file']
Dropped column: source_file
Updated feature shape: (2012223, 78)
Converted features to float32.
Categorical columns: []


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

print("Feature scaling completed.")
print("Mean (first 5 features):", X_scaled.mean(axis=0)[:5])
print("Std (first 5 features):", X_scaled.std(axis=0)[:5])


np.save(os.path.join(PROCESSED_DATA_DIR, "X_scaled.npy"), X_scaled)
np.save(os.path.join(PROCESSED_DATA_DIR, "y.npy"), y.values)

print("Scaled features and labels saved.")

Feature scaling completed.
Mean (first 5 features): [-1.5188851e-08 -3.7338943e-08  6.9669276e-10  1.0995423e-09
  4.4076480e-09]
Std (first 5 features): [0.9999998 0.9999999 0.9999998 1.0000001 1.0000002]
Scaled features and labels saved.


In [ ]:
import joblib

scaler_path = os.path.join(PROCESSED_DATA_DIR, "standard_scaler.pkl")
joblib.dump(scaler, scaler_path)

print("Scaler saved at:", scaler_path)

assert X_scaled.shape[0] == y.shape[0], "Mismatch in X and y sizes!"
assert not np.isnan(X_scaled).any(), "NaN values found!"
assert not np.isinf(X_scaled).any(), "Infinity values found!"

print("Phase 2 completed successfully.")


Scaler saved at: /content/drive/MyDrive/FYP_FL_IDS/data/processed/standard_scaler.pkl
Phase 2 completed successfully.


In [ ]:
X_path = os.path.join(PROCESSED_DATA_DIR, "X_scaled.npy")
y_path = os.path.join(PROCESSED_DATA_DIR, "y.npy")

X_scaled = np.load(X_path, mmap_mode="r")
y = np.load(y_path, mmap_mode="r")

print("Loaded X_scaled (memmap):", X_scaled.shape)
print("Loaded y (memmap):", y.shape)

SEQ_LEN = CONFIG["SEQUENCE_LENGTH"]
print("Sequence length:", SEQ_LEN)

CHUNK_SIZE = 200_000

TOTAL_ROWS = X_scaled.shape[0]
NUM_FEATURES = X_scaled.shape[1]

print("Total rows:", TOTAL_ROWS)
print("Chunk size:", CHUNK_SIZE)
print("Features:", NUM_FEATURES)


Loaded X_scaled (memmap): (2012223, 78)
Loaded y (memmap): (2012223,)
Sequence length: 10
Total rows: 2012223
Chunk size: 200000
Features: 78


In [ ]:
def process_chunk1(X, y, start_idx, end_idx, seq_len):
    X_seq = []
    y_seq = []

    for i in range(start_idx, end_idx - seq_len + 1):
        X_seq.append(X[i:i + seq_len])
        y_seq.append(y[i + seq_len - 1])

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.int64)

def process_chunk(X, y, start_idx, end_idx, seq_len, stride=5):
    X_seq = []
    y_seq = []

    for i in range(start_idx, end_idx - seq_len + 1, stride):
        X_seq.append(X[i:i + seq_len])
        y_seq.append(y[i + seq_len - 1])

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.int64)


In [ ]:
import shutil

output_dir = os.path.join(PROCESSED_DATA_DIR, "sequence_chunks")

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)

os.makedirs(output_dir)
print("sequence_chunks cleared.")


sequence_chunks cleared.


In [ ]:
output_dir = os.path.join(PROCESSED_DATA_DIR, "sequence_chunks")
os.makedirs(output_dir, exist_ok=True)

chunk_id = 0

for start in range(0, TOTAL_ROWS - SEQ_LEN + 1, CHUNK_SIZE):
    end = min(start + CHUNK_SIZE + SEQ_LEN - 1, TOTAL_ROWS)

    print(f"\nProcessing chunk {chunk_id} | rows {start} to {end}")

    X_chunk_seq, y_chunk_seq = process_chunk(
        X_scaled, y, start, end, SEQ_LEN, stride=5
    )

    np.save(os.path.join(output_dir, f"X_seq_chunk_{chunk_id}.npy"), X_chunk_seq)
    np.save(os.path.join(output_dir, f"y_seq_chunk_{chunk_id}.npy"), y_chunk_seq)

    print("Saved shapes:", X_chunk_seq.shape, y_chunk_seq.shape)

    del X_chunk_seq, y_chunk_seq  # free RAM
    chunk_id += 1

print("\nAll chunks processed successfully.")



Processing chunk 0 | rows 0 to 200009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 1 | rows 200000 to 400009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 2 | rows 400000 to 600009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 3 | rows 600000 to 800009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 4 | rows 800000 to 1000009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 5 | rows 1000000 to 1200009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 6 | rows 1200000 to 1400009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 7 | rows 1400000 to 1600009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 8 | rows 1600000 to 1800009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 9 | rows 1800000 to 2000009
Saved shapes: (40000, 10, 78) (40000,)

Processing chunk 10 | rows 2000000 to 2012223
Saved shapes: (2443, 10, 78) (2443,)

All chunks processed successfully.


In [ ]:
chunk_files = sorted(os.listdir(output_dir))
print("Total chunks created:", len(chunk_files))
print("Sample files:", chunk_files[:4])

total_sequences = 0

for f in chunk_files:
    if f.startswith("y_seq"):
        y_chunk = np.load(os.path.join(output_dir, f))
        total_sequences += len(y_chunk)

print("Total sequences created:", total_sequences)
print("Expected:", TOTAL_ROWS - SEQ_LEN + 1)


Total chunks created: 22
Sample files: ['X_seq_chunk_0.npy', 'X_seq_chunk_1.npy', 'X_seq_chunk_10.npy', 'X_seq_chunk_2.npy']
Total sequences created: 402443
Expected: 2012214


In [ ]:
import os
import numpy as np

SEQ_CHUNK_DIR = os.path.join(PROCESSED_DATA_DIR, "sequence_chunks")

print("Sequence chunk directory exists:", os.path.exists(SEQ_CHUNK_DIR))

files = sorted(os.listdir(SEQ_CHUNK_DIR))
print("Total files:", len(files))
print("Sample files:", files[:6])

total_sequences = 0

for f in files:
    if f.startswith("y_seq_chunk"):
        y_chunk = np.load(os.path.join(SEQ_CHUNK_DIR, f), mmap_mode="r")
        total_sequences += len(y_chunk)

print("Total sequences found:", total_sequences)


Sequence chunk directory exists: True
Total files: 22
Sample files: ['X_seq_chunk_0.npy', 'X_seq_chunk_1.npy', 'X_seq_chunk_10.npy', 'X_seq_chunk_2.npy', 'X_seq_chunk_3.npy', 'X_seq_chunk_4.npy']
Total sequences found: 402443


Phase 4

In [ ]:
import shutil

CLIENT_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, "federated_clients")

if os.path.exists(CLIENT_DATA_DIR):
    shutil.rmtree(CLIENT_DATA_DIR)

os.makedirs(CLIENT_DATA_DIR)

for c in CLIENTS:
    os.makedirs(os.path.join(CLIENT_DATA_DIR, c))

print("federated_clients cleared.")


federated_clients cleared.


In [ ]:
import os
import numpy as np

SEQ_CHUNK_DIR = os.path.join(PROCESSED_DATA_DIR, "sequence_chunks")
assert os.path.exists(SEQ_CHUNK_DIR), "sequence_chunks directory not found"

x_chunks = sorted([f for f in os.listdir(SEQ_CHUNK_DIR) if f.startswith("X_seq_chunk")])
y_chunks = sorted([f for f in os.listdir(SEQ_CHUNK_DIR) if f.startswith("y_seq_chunk")])

assert len(x_chunks) == len(y_chunks), "Mismatch between X and y chunks"

NUM_CHUNKS = len(x_chunks)
print("Total sequence chunks:", NUM_CHUNKS)


Total sequence chunks: 11


In [ ]:
CLIENTS = ["Bank_A", "Bank_B", "Bank_C"]
NUM_CLIENTS = len(CLIENTS)

CLIENT_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, "federated_clients")
os.makedirs(CLIENT_DATA_DIR, exist_ok=True)

for c in CLIENTS:
    os.makedirs(os.path.join(CLIENT_DATA_DIR, c), exist_ok=True)

print("Federated clients initialized:", CLIENTS)


Federated clients initialized: ['Bank_A', 'Bank_B', 'Bank_C']


In [ ]:
DATA_DISTRIBUTION = "iid"
# options: "iid", "non_iid_label_skew"


In [ ]:
import random

def iid_partition(num_chunks, clients):
    chunk_ids = list(range(num_chunks))
    random.shuffle(chunk_ids)

    assignments = {c: [] for c in clients}
    for idx, cid in enumerate(chunk_ids):
        assignments[clients[idx % len(clients)]].append(cid)

    return assignments

def non_iid_label_skew_partition(y_chunks, clients, dominance=0.7):

    client_assignments = {c: [] for c in clients}

    # analyze chunk labels
    attack_chunks = []
    benign_chunks = []

    for i, y_file in enumerate(y_chunks):
        y = np.load(os.path.join(SEQ_CHUNK_DIR, y_file), mmap_mode="r")
        attack_ratio = y.mean()  # since attack=1, benign=0

        if attack_ratio > 0.5:
            attack_chunks.append(i)
        else:
            benign_chunks.append(i)

    random.shuffle(attack_chunks)
    random.shuffle(benign_chunks)

    for idx, client in enumerate(clients):
        num_attack = int(dominance * len(attack_chunks) / len(clients))
        num_benign = int((1 - dominance) * len(benign_chunks) / len(clients))

        client_assignments[client].extend(attack_chunks[:num_attack])
        client_assignments[client].extend(benign_chunks[:num_benign])

        del attack_chunks[:num_attack]
        del benign_chunks[:num_benign]

    return client_assignments
if DATA_DISTRIBUTION == "iid":
    client_assignments = iid_partition(NUM_CHUNKS, CLIENTS)

elif DATA_DISTRIBUTION == "non_iid_label_skew":
    client_assignments = non_iid_label_skew_partition(y_chunks, CLIENTS)

else:
    raise ValueError("Unknown DATA_DISTRIBUTION")


In [ ]:
import shutil

for client, chunk_ids in client_assignments.items():
    client_path = os.path.join(CLIENT_DATA_DIR, client)

    for cid in chunk_ids:
        shutil.copy(
            os.path.join(SEQ_CHUNK_DIR, x_chunks[cid]),
            client_path
        )
        shutil.copy(
            os.path.join(SEQ_CHUNK_DIR, y_chunks[cid]),
            client_path
        )

print("Client-local datasets created successfully.")


Client-local datasets created successfully.


In [ ]:
import json

metadata = {
    "clients": CLIENTS,
    "num_chunks": NUM_CHUNKS,
    "data_distribution": DATA_DISTRIBUTION,
    "client_assignments": client_assignments
}

meta_path = os.path.join(CLIENT_DATA_DIR, "federated_metadata.json")

with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=4)

print("Federated metadata saved at:", meta_path)


Federated metadata saved at: /content/drive/MyDrive/FYP_FL_IDS/data/processed/federated_clients/federated_metadata.json


In [ ]:
for c in CLIENTS:
    files = os.listdir(os.path.join(CLIENT_DATA_DIR, c))
    x_files = [f for f in files if f.startswith("X_seq")]
    y_files = [f for f in files if f.startswith("y_seq")]

    print(f"{c}: X_chunks={len(x_files)}, y_chunks={len(y_files)}")


Bank_A: X_chunks=4, y_chunks=4
Bank_B: X_chunks=4, y_chunks=4
Bank_C: X_chunks=3, y_chunks=3


Phase 5

In [ ]:
import os
import numpy as np

SEQ_CHUNK_DIR = os.path.join(PROCESSED_DATA_DIR, "sequence_chunks")


sample_path = os.path.join(SEQ_CHUNK_DIR, "X_seq_chunk_0.npy")
X_sample = np.load(sample_path, mmap_mode="r")

SEQ_LEN = CONFIG["SEQUENCE_LENGTH"]
NUM_FEATURES = X_sample.shape[2]

print("Sequence length:", SEQ_LEN)
print("Number of features:", NUM_FEATURES)


Sequence length: 10
Number of features: 78


In [ ]:
class CNN_LSTM_IDS(nn.Module):
    def __init__(self, seq_len, num_features):
        super().__init__()

        # Spatial feature extraction
        self.conv1 = nn.Conv1d(
            in_channels=num_features,
            out_channels=64,
            kernel_size=3,
            padding=1
        )
        self.relu = nn.ReLU()

        # Temporal modeling
        self.lstm = nn.LSTM(
            input_size=64,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )

        # Classifier
        self.fc = nn.Linear(64, 1)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # x: (B, T, F) -> Conv1d expects (B, F, T)
        x = x.permute(0, 2, 1)
        x = self.relu(self.conv1(x))
        # back to (B, T, C)
        x = x.permute(0, 2, 1)

        _, (h_n, _) = self.lstm(x)
        h_last = h_n[-1]           # (B, 64)
        out = self.fc(h_last)
        return out


In [ ]:
cnn_lstm_model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)
print("CNN-LSTM parameters:", count_parameters(cnn_lstm_model))
# Create a tiny dummy batch
BATCH_TEST = 4
dummy_x = torch.randn(BATCH_TEST, SEQ_LEN, NUM_FEATURES, device=DEVICE)

with torch.no_grad():
   # y_dnn = dnn_model(dummy_x)
    y_cnnlstm = cnn_lstm_model(dummy_x)

#print("DNN output shape:", y_dnn.shape)
print("CNN-LSTM output shape:", y_cnnlstm.shape)


CNN-LSTM parameters: 48385
CNN-LSTM output shape: torch.Size([4, 1])


In [ ]:
os.makedirs(MODELS_DIR, exist_ok=True)

#torch.save(dnn_model.state_dict(), os.path.join(MODELS_DIR, "dnn_init.pt"))
#torch.save(cnn_lstm_model.state_dict(), os.path.join(MODELS_DIR, "cnn_lstm_init.pt"))
init_model_path = os.path.join(
    MODELS_DIR, f"cnn_lstm_init_{MODEL_VERSION}.pt"
)

torch.save(cnn_lstm_model.state_dict(), init_model_path)

print("Initial model weights saved.")


Initial model weights saved.


In [ ]:
import json

model_meta = {
    "sequence_length": SEQ_LEN,
    "num_features": NUM_FEATURES,
    #"dnn_params": count_parameters(dnn_model),
    "cnn_lstm_params": count_parameters(cnn_lstm_model)
}

meta_path = os.path.join(MODELS_DIR, "model_metadata.json")
with open(meta_path, "w") as f:
    json.dump(model_meta, f, indent=4)

print("Model metadata saved:", meta_path)


Model metadata saved: /content/drive/MyDrive/FYP_FL_IDS/models/model_metadata.json


phase 6

## Phase 6: FedPHE - Packed Homomorphic Encryption Setup

This phase implements **FedPHE (Federated Learning with Packed Homomorphic Encryption)** using TenSEAL.



### Load and Evaluate Your Custom Saved Model

First, define the path to your saved model and load its state dictionary into a new instance of the `CNN_LSTM_IDS` model.

In [ ]:
from torch.utils.data import Dataset
import numpy as np
import os
import torch

class ClientSequenceDataset(Dataset):
    def __init__(self, client_dir):
        self.x_files = sorted([
            os.path.join(client_dir, f)
            for f in os.listdir(client_dir) if f.startswith("X_seq")
        ])
        self.y_files = sorted([
            os.path.join(client_dir, f)
            for f in os.listdir(client_dir) if f.startswith("y_seq")
        ])

        assert len(self.x_files) == len(self.y_files)

        self.chunk_sizes = []
        for yf in self.y_files:
            y = np.load(yf, mmap_mode="r")
            self.chunk_sizes.append(len(y))

        self.cumulative_sizes = np.cumsum(self.chunk_sizes)


        self._current_chunk_id = None
        self._current_x = None
        self._current_y = None

    def __len__(self):
        return int(self.cumulative_sizes[-1])

    def __getitem__(self, idx):
        chunk_id = np.searchsorted(self.cumulative_sizes, idx, side="right")
        local_idx = idx if chunk_id == 0 else idx - self.cumulative_sizes[chunk_id - 1]


        if chunk_id != self._current_chunk_id:
            self._current_x = np.load(self.x_files[chunk_id], mmap_mode="r")
            self._current_y = np.load(self.y_files[chunk_id], mmap_mode="r")
            self._current_chunk_id = chunk_id

        x = self._current_x[local_idx]
        y = self._current_y[local_idx]

        return (
            torch.tensor(x, dtype=torch.float32),
            torch.tensor(y, dtype=torch.float32)
        )



In [ ]:
MAX_BATCHES = 50

def local_train(model, dataloader, epochs, lr):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = CRITERION

    for epoch in range(epochs):
        for batch_idx, (x, y) in enumerate(dataloader):
            if batch_idx >= MAX_BATCHES:
                break

            x = x.to(DEVICE)
            y = y.to(DEVICE).unsqueeze(1)

            optimizer.zero_grad()
            preds = model(x)
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step()

    return model.state_dict()



def fedavg(state_dicts, data_sizes):
    avg_state = {}
    total = sum(data_sizes)

    for key in state_dicts[0].keys():
        avg_state[key] = sum(
            state_dicts[i][key].float() * data_sizes[i] / total
            for i in range(len(state_dicts))
        )

    return avg_state



In [ ]:
global_model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)

init_path = os.path.join(MODELS_DIR, "cnn_lstm_init.pt")
global_model.load_state_dict(torch.load(init_path))

print("Global CNN-LSTM model initialized.")


Global CNN-LSTM model initialized.


In [ ]:
from torch.utils.data import DataLoader

CLIENT_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, "federated_clients")
CLIENTS = ["Bank_A", "Bank_B", "Bank_C"]

client_loaders = {}
client_sizes = {}

for client in CLIENTS:
    dataset = ClientSequenceDataset(
        os.path.join(CLIENT_DATA_DIR, client)
    )

    loader = DataLoader(
        dataset,
        batch_size=CONFIG["BATCH_SIZE"],
        shuffle=True,
        num_workers=0,
        pin_memory=False
    )

    client_loaders[client] = loader
    client_sizes[client] = len(dataset)

    print(f"{client} samples:", len(dataset))


Bank_A samples: 160000
Bank_B samples: 160000
Bank_C samples: 82443


In [ ]:

eval_dataset = ClientSequenceDataset(
    os.path.join(CLIENT_DATA_DIR, "Bank_A")
)

EVAL_SAMPLES = 5000  # keep small
eval_indices = np.random.choice(len(eval_dataset), EVAL_SAMPLES, replace=False)

eval_subset = torch.utils.data.Subset(eval_dataset, eval_indices)

eval_loader = DataLoader(
    eval_subset,
    batch_size=256,
    shuffle=False,
    num_workers=0
)
from sklearn.metrics import (
    accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score
)

def evaluate_global_model(model, dataloader, threshold=0.3):
    model.eval()

    all_probs = []
    all_labels = []

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)

            logits = model(x)
            probs = torch.sigmoid(logits).cpu().numpy().ravel()

            all_probs.extend(probs)
            all_labels.extend(y.cpu().numpy())  # 🔑 FIX

    all_probs = np.array(all_probs)
    all_labels = np.array(all_labels)

    preds = (all_probs > threshold).astype(int)

    metrics = {
        "accuracy": accuracy_score(all_labels, preds),
        "precision": precision_score(all_labels, preds, zero_division=0),
        "recall": recall_score(all_labels, preds, zero_division=0),
        "f1": f1_score(all_labels, preds, zero_division=0),
        "roc_auc": roc_auc_score(all_labels, all_probs)
    }

    return metrics


In [ ]:
import tenseal as ts

def create_ckks_context():
    ctx = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    ctx.global_scale = 2**30
    ctx.generate_galois_keys()
    return ctx


ckks_ctx = create_ckks_context()


public_ctx = ckks_ctx.copy()
public_ctx.make_context_public()


def compute_model_update(local_model, global_model):
    delta = {}
    global_state = global_model.state_dict()
    local_state = local_model.state_dict()

    for k in global_state:
        if ("lstm" in k or "fc" in k) and ("bias" not in k):
            delta[k] = local_state[k] - global_state[k]
    return delta

def encrypt_update(delta_state, ctx):
    encrypted = []
    shapes = []

    for k, tensor in delta_state.items():
        arr = tensor.detach().cpu().numpy()
        shapes.append((k, arr.shape))
        enc = ts.ckks_vector(ctx, arr.flatten())
        encrypted.append(enc)

    return encrypted, shapes
def encrypted_sum(encrypted_updates):
    agg = encrypted_updates[0]
    for i in range(1, len(encrypted_updates)):
        agg = [a + b for a, b in zip(agg, encrypted_updates[i])]
    return agg


def decrypt_update(enc_agg, shapes):
    delta = {}
    for enc, (k, shape) in zip(enc_agg, shapes):
        dec = np.array(enc.decrypt())
        delta[k] = torch.tensor(dec.reshape(shape), dtype=torch.float32)
    return delta


In [ ]:

import tenseal as ts
import numpy as np
import torch


HE_POLY_MODULUS = 16384
HE_SCALE_BITS = 40
HE_COEFF_MOD_BITS = [60, 40, 40, 40, 40, 60]

# Selected layers to encrypt (60% faster)
SELECTED_LAYERS = [
    "lstm.weight_ih_l0",
    "lstm.weight_hh_l0",
    "fc.weight",
    "fc.bias"
]

# ============================================================================
# CREATE CKKS CONTEXT
# ============================================================================

def create_ckks_context():
    """Creates CKKS context - call ONCE before training"""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=HE_POLY_MODULUS,
        coeff_mod_bit_sizes=HE_COEFF_MOD_BITS
    )
    context.global_scale = 2 ** HE_SCALE_BITS
    context.generate_galois_keys()
    return context

# ============================================================================
# COMPUTE MODEL UPDATE (ΔW)
# ============================================================================

def compute_model_update(local_model, global_model):
    """Computes ΔW = W_local - W_global"""
    delta = {}
    local_state = local_model.state_dict()
    global_state = global_model.state_dict()

    for key in SELECTED_LAYERS:
        if key in local_state and key in global_state:
            diff = local_state[key] - global_state[key]
            # Clip extreme values
            diff = torch.clamp(diff, min=-10.0, max=10.0)
            delta[key] = diff

    return delta

# ============================================================================
# ENCRYPT UPDATE
# ============================================================================

def encrypt_update(delta, context):
    """Encrypts weight delta"""
    encrypted = {}
    shapes = {}

    for key, tensor in delta.items():
        shapes[key] = tensor.shape
        flat = tensor.cpu().detach().numpy().flatten()

        # Validate
        if np.isnan(flat).any() or np.isinf(flat).any():
            flat = np.nan_to_num(flat, nan=0.0, posinf=0.0, neginf=0.0)

        # Clip
        flat = np.clip(flat, -10.0, 10.0)

        # Encrypt
        encrypted[key] = ts.ckks_vector(context, flat.tolist())

    return encrypted, shapes

# ============================================================================
# ENCRYPTED SUM (SERVER-SIDE AGGREGATION)
# ============================================================================

def encrypted_sum(encrypted_updates):
    """Sums encrypted updates from all clients"""
    if not encrypted_updates:
        return {}

    result = {}
    all_keys = encrypted_updates[0].keys()

    for key in all_keys:
        # Start with first client
        result[key] = encrypted_updates[0][key]

        # Add remaining clients
        for i in range(1, len(encrypted_updates)):
            result[key] = result[key] + encrypted_updates[i][key]

    return result

# ============================================================================
# DECRYPT UPDATE
# ============================================================================

def decrypt_update(encrypted_sum, shapes):
    """Decrypts aggregated update"""
    decrypted = {}

    for key, enc_vec in encrypted_sum.items():
        # Decrypt
        flat = enc_vec.decrypt()
        flat = np.array(flat, dtype=np.float32)

        # Validate
        flat = np.nan_to_num(flat, nan=0.0, posinf=0.0, neginf=0.0)

        # Reshape
        shape = shapes[key]
        num_elements = np.prod(shape)
        flat = flat[:num_elements]

        tensor = torch.tensor(flat, dtype=torch.float32)
        tensor = tensor.reshape(shape)

        decrypted[key] = tensor

    return decrypted

# ============================================================================
# USAGE IN YOUR TRAINING LOOP:
# ============================================================================

# BEFORE training loop:
# ckks_ctx = create_ckks_context()

# INSIDE your loop - NO CHANGES NEEDED to your code!
# Your code already calls these functions correctly.

In [ ]:
import os
import torch

CHECKPOINT_DIR = os.path.join(BASE_DIR, "fedphe_checkpoints1")
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def save_checkpoint(round_idx, model):
    ckpt_path = os.path.join(CHECKPOINT_DIR, f"fedphe_round_{round_idx}.pt")
    torch.save({
        "round": round_idx,
        "model_state": model.state_dict()
    }, ckpt_path)
    print(f" Checkpoint saved: {ckpt_path}")


def load_latest_checkpoint(model):
    if not os.path.exists(CHECKPOINT_DIR):
        return 0  # start from scratch

    ckpts = [f for f in os.listdir(CHECKPOINT_DIR) if f.endswith(".pt")]
    if not ckpts:
        return 0

    ckpts.sort(key=lambda x: int(x.split("_")[-1].split(".")[0]))
    latest_ckpt = ckpts[-1]

    ckpt_path = os.path.join(CHECKPOINT_DIR, latest_ckpt)
    data = torch.load(ckpt_path, map_location=DEVICE)

    model.load_state_dict(data["model_state"])
    start_round = data["round"] + 1

    print(f" Resuming from checkpoint: {ckpt_path}")
    print(f"  Starting from round {start_round+1}")

    return start_round


In [ ]:
ckks_ctx = create_ckks_context()
ROUNDS = CONFIG["ROUNDS"]

start_round = load_latest_checkpoint(global_model)

for rnd in range(start_round, ROUNDS):
    print(f"\n===== Federated Round {rnd+1}/{ROUNDS} =====")

    encrypted_updates = []


    for client in CLIENTS:
        print(f"Training locally on {client}...")

        local_model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)
        local_model.load_state_dict(global_model.state_dict())

        local_train(
            local_model,
            client_loaders[client],
            CONFIG["LOCAL_EPOCHS"],
            CONFIG["LEARNING_RATE"]
        )

        delta = compute_model_update(local_model, global_model)
        enc_delta, shapes = encrypt_update(delta, ckks_ctx)
        encrypted_updates.append(enc_delta)

        del local_model
        torch.cuda.empty_cache()


    enc_sum = encrypted_sum(encrypted_updates)


    delta_avg = decrypt_update(enc_sum, shapes)

    for k in delta_avg:
        delta_avg[k] /= len(CLIENTS)

    if rnd == 0:
        norm = sum(torch.norm(v).item() for v in delta_avg.values())
        print("ΔW norm (sanity):", norm)

    current_state = global_model.state_dict()
    for k in delta_avg:
        current_state[k] += delta_avg[k].to(DEVICE)

    global_model.load_state_dict(current_state)

    print("Global model updated.")


    CHECKPOINT_FREQ = 2
    if (rnd + 1) % CHECKPOINT_FREQ == 0:
        save_checkpoint(rnd, global_model)

    import gc
    gc.collect()


🔁 Resuming from checkpoint: /content/drive/MyDrive/FYP_FL_IDS/fedphe_checkpoints1/fedphe_round_19.pt
➡️  Starting from round 21

===== Federated Round 21/25 =====
Training locally on Bank_A...
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
Training locally on Bank_B...
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.
The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increas

In [ ]:
#OLD

ckks_ctx = create_ckks_context()
ROUNDS = CONFIG["ROUNDS"]

for rnd in range(ROUNDS):
    print(f"\n===== Federated Round {rnd+1}/{ROUNDS} =====")

    encrypted_updates = []   # 🔐 collect ALL client updates

    # -------- CLIENT SIDE --------
    for client in CLIENTS:
        print(f"Training locally on {client}...")

        local_model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)
        local_model.load_state_dict(global_model.state_dict())

        local_train(
            local_model,
            client_loaders[client],
            CONFIG["LOCAL_EPOCHS"],
            CONFIG["LEARNING_RATE"]
        )

        # 🔐 compute & encrypt UPDATE (ΔW)
        delta = compute_model_update(local_model, global_model)
        enc_delta, shapes = encrypt_update(delta, ckks_ctx)

        encrypted_updates.append(enc_delta)

        del local_model
        torch.cuda.empty_cache()

    # -------- SERVER SIDE (BLIND) --------
    enc_sum = encrypted_sum(encrypted_updates)

    # -------- CLIENT SIDE (DECRYPT + APPLY) --------
    delta_avg = decrypt_update(enc_sum, shapes)

    # Average
    for k in delta_avg:
        delta_avg[k] /= len(CLIENTS)

    # 🔎 Sanity check (only once)
    if rnd == 0:
        norm = sum(torch.norm(v).item() for v in delta_avg.values())
        print("ΔW norm (sanity):", norm)

    # Apply update
    current_state = global_model.state_dict()
    for k in delta_avg:
        current_state[k] += delta_avg[k].to(DEVICE)

    global_model.load_state_dict(current_state)

    print("Global model updated.")

    import gc
    gc.collect()


NameError: name 'create_ckks_context' is not defined

In [ ]:
final_model_path = os.path.join(
    MODELS_DIR, f"cnn_lstm_global_with_HE_25rounds_16k.pt"
)

torch.save(global_model.state_dict(), final_model_path)




print("Plaintext FL model saved at:", final_model_path)


Plaintext FL model saved at: /content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_25rounds_16k.pt


In [ ]:
import torch
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate model (MUST match training architecture)
model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)

# Load checkpoint or final model
MODEL_PATH = "/content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_25rounds_16k.pt"
# OR checkpoint: fedphe_checkpoints/fedphe_round_8.pt

state = torch.load(MODEL_PATH, map_location=DEVICE)

# Handle checkpoint vs final model
if isinstance(state, dict) and "model_state" in state:
    model.load_state_dict(state["model_state"])
else:
    model.load_state_dict(state)

model.eval()
print("✅ FedPHE model loaded for evaluation")


✅ FedPHE model loaded for evaluation


In [ ]:
def evaluate_model(model, dataloader, threshold=0.3):
    y_true, y_pred, y_prob = [], [], []

    model.eval()
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.cpu().numpy()

            logits = model(x)
            probs = torch.sigmoid(logits).cpu().numpy().flatten()
            preds = (probs > threshold).astype(int)

            y_true.extend(y)
            y_pred.extend(preds)
            y_prob.extend(probs)

    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_prob),
        "confusion_matrix": confusion_matrix(y_true, y_pred)
    }

    return metrics


In [ ]:
THRESHOLD = 0.5 # good for IDS demos

metrics = evaluate_model(model, eval_loader, threshold=THRESHOLD)

print("\n📊 FedPHE Evaluation Results")
print("="*40)
print(f"Accuracy:  {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall:    {metrics['recall']:.4f}")
print(f"F1-score:  {metrics['f1']:.4f}")
print(f"ROC-AUC:   {metrics['roc_auc']:.4f}")
print("\nConfusion Matrix:")
print(metrics["confusion_matrix"])
print("="*40)



📊 FedPHE Evaluation Results
Accuracy:  0.9782
Precision: 0.9838
Recall:    0.9522
F1-score:  0.9678
ROC-AUC:   0.9958

Confusion Matrix:
[[104158    861]
 [  2626  52355]]


In [ ]:
import torch

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_saved_model(model_path, model_class, seq_len, num_features):
    """
    model_path: path to .pt file
    model_class: CNN_LSTM_IDS
    """
    model = model_class(seq_len, num_features).to(DEVICE)

    state = torch.load(model_path, map_location=DEVICE)

    # Handle checkpoint or plain model
    if isinstance(state, dict) and "model_state" in state:
        model.load_state_dict(state["model_state"])
    else:
        model.load_state_dict(state)

    model.eval()
    print(f"✅ Loaded model: {model_path}")
    return model


In [ ]:
import numpy as np

SEQ_LEN = 10
NUM_FEATURES = 78  # FIXED

benign_normal = np.random.normal(
    loc=0.05,
    scale=0.05,
    size=(SEQ_LEN, NUM_FEATURES)
)

attack_ddos = np.random.normal(
    loc=1.2,
    scale=0.8,
    size=(SEQ_LEN, NUM_FEATURES)
)

# burst spikes (packet count, byte count, flow rate)
attack_ddos[:, :6] += 3.5
attack_ddos[:, 12:18] += 2.0

attack_slow = np.random.normal(
    loc=0.4,
    scale=0.25,
    size=(SEQ_LEN, NUM_FEATURES)
)

# temporal accumulation
attack_slow = np.cumsum(attack_slow, axis=0)

# protocol misuse
attack_slow[:, 20:26] += 1.5

attack_portscan = np.random.normal(
    loc=0.3,
    scale=0.6,
    size=(SEQ_LEN, NUM_FEATURES)
)

# many short-lived flows
attack_portscan[:, 30:40] += 2.5
attack_portscan[:, 5:10] -= 0.2

attack_bruteforce = np.random.normal(
    loc=0.5,
    scale=0.3,
    size=(SEQ_LEN, NUM_FEATURES)
)

# repeated authentication failures
attack_bruteforce[:, 45:50] += 3.0
attack_bruteforce[:, 0:2] += 1.2

attack_botnet = np.random.normal(
    loc=0.7,
    scale=0.15,
    size=(SEQ_LEN, NUM_FEATURES)
)

# periodic beacons
for t in range(SEQ_LEN):
    attack_botnet[t, 60:65] += (t % 2) * 2.5

attack_exfiltration = np.random.normal(
    loc=0.4,
    scale=0.2,
    size=(SEQ_LEN, NUM_FEATURES)
)

# sustained payload size
attack_exfiltration[:, 70:75] += 2.8
attack_exfiltration[:, 15:18] += 1.2

attack_hybrid = np.random.normal(
    loc=0.6,
    scale=0.5,
    size=(SEQ_LEN, NUM_FEATURES)
)

# combine behaviors
attack_hybrid[:, :5] += 2.5         # burst
attack_hybrid[:, 20:25] += 1.5      # protocol anomaly
attack_hybrid[:, 45:50] += 2.0      # auth failures

def test_all_attacks(model, threshold=0.5):
    attacks = {
        "Benign": benign_normal,
        "DDoS": attack_ddos,
        "Slow Attack": attack_slow,
        "Port Scan": attack_portscan,
        "Brute Force": attack_bruteforce,
        "Botnet": attack_botnet,
        "Exfiltration": attack_exfiltration,
        "Hybrid": attack_hybrid,
    }

    print("="*60)
    for name, sample in attacks.items():
        prob, pred = predict_sample(sample, model, threshold)
        print(f"{name:<15} → {pred:<10} | prob={prob:.4f}")
    print("="*60)



In [ ]:
def predict_sample(sample, model, threshold=0.3):
    x = torch.tensor(sample, dtype=torch.float32).unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logit = model(x)
        prob = torch.sigmoid(logit).item()

    label = "ATTACK 🚨" if prob > threshold else "BENIGN ✅"
    return prob, label


In [ ]:
MODEL_PATHS = {
    "HE_10_rounds_16k":"/content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_12rounds_16k.pt",
    "16k20 rounds":"/content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_20rounds_16k.pt",
    "HE_25_rounds_16k": "/content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_25rounds_16k.pt",
}


In [ ]:
THRESHOLD = 0.5

for name, path in MODEL_PATHS.items():
    print("\n" + "="*60)
    print(f"🔍 Testing model: {name}")
    print("="*60)

    model = load_saved_model(
        path,
        CNN_LSTM_IDS,
        SEQ_LEN,
        NUM_FEATURES
    )
    test_all_attacks(model, threshold=THRESHOLD)




🔍 Testing model: HE_10_rounds_16k
✅ Loaded model: /content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_12rounds_16k.pt
Benign          → BENIGN ✅   | prob=0.1442
DDoS            → ATTACK 🚨   | prob=0.8618
Slow Attack     → ATTACK 🚨   | prob=0.8605
Port Scan       → BENIGN ✅   | prob=0.1425
Brute Force     → BENIGN ✅   | prob=0.0850
Botnet          → ATTACK 🚨   | prob=0.7993
Exfiltration    → BENIGN ✅   | prob=0.4321
Hybrid          → BENIGN ✅   | prob=0.0072

🔍 Testing model: 16k20 rounds
✅ Loaded model: /content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_20rounds_16k.pt
Benign          → BENIGN ✅   | prob=0.0951
DDoS            → ATTACK 🚨   | prob=0.7983
Slow Attack     → ATTACK 🚨   | prob=0.7965
Port Scan       → BENIGN ✅   | prob=0.1303
Brute Force     → BENIGN ✅   | prob=0.1648
Botnet          → ATTACK 🚨   | prob=0.8854
Exfiltration    → ATTACK 🚨   | prob=0.7894
Hybrid          → BENIGN ✅   | prob=0.0052

🔍 Testing model: HE_25_rounds_16k
✅ Loaded model: /

In [ ]:
def predict_single_sample(sample, model, threshold=0.3):
    """
    sample: Tensor or numpy array of shape (SEQ_LEN, NUM_FEATURES)
    """
    model.eval()

    if isinstance(sample, np.ndarray):
        sample = torch.tensor(sample, dtype=torch.float32)

    x = sample.unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        logit = model(x)
        prob = torch.sigmoid(logit).item()

    label = "ATTACK 🚨" if prob > threshold else "BENIGN ✅"
    return prob, label


sample_x, sample_y = next(iter(eval_loader))
sample_x = sample_x[0]
sample_y = sample_y[0].item()

prob, label = predict_single_sample(sample_x, model, threshold=THRESHOLD)

print("\n🚦 Demo Prediction")
print("="*30)
print("Ground Truth:", "ATTACK" if sample_y == 1 else "BENIGN")
print(f"Attack Probability: {prob:.4f}")
print("Prediction:", label)
print("="*30)




🚦 Demo Prediction
Ground Truth: ATTACK
Attack Probability: 0.9888
Prediction: ATTACK 🚨


In [ ]:
import torch

init_weights = torch.load(
    os.path.join(MODELS_DIR, "cnn_lstm_init.pt")
)
final_weights = global_model.state_dict()

diff = 0.0
for k in init_weights:
    diff += torch.norm(init_weights[k] - final_weights[k]).item()

print("Total weight change:", diff)


Total weight change: 11.202270030975342


In [ ]:
from torch.utils.data import DataLoader

# Use Bank_A as evaluation source
EVAL_CLIENT = "Bank_A"
EVAL_DIR = os.path.join(PROCESSED_DATA_DIR, "federated_clients", EVAL_CLIENT)

eval_dataset = ClientSequenceDataset(EVAL_DIR)

# Limit evaluation size (important for speed)
EVAL_SAMPLES = 5000

eval_loader = DataLoader(
    eval_dataset,
    batch_size=128,
    shuffle=False
)

print("Evaluation samples available:", len(eval_dataset))
print("Evaluation samples used:", EVAL_SAMPLES)


Evaluation samples available: 160000
Evaluation samples used: 5000


In [ ]:
import numpy as np

# Load ONLY labels (fast)
y_indices_0 = []
y_indices_1 = []

max_per_class = 1000  # reduce further if needed

seen = 0

for y_file in eval_dataset.y_files:
    y_chunk = np.load(y_file, mmap_mode="r")

    for i, label in enumerate(y_chunk):
        global_idx = seen + i

        if label == 0 and len(y_indices_0) < max_per_class:
            y_indices_0.append(global_idx)
        elif label == 1 and len(y_indices_1) < max_per_class:
            y_indices_1.append(global_idx)

        if len(y_indices_0) >= max_per_class and len(y_indices_1) >= max_per_class:
            break

    seen += len(y_chunk)
    if len(y_indices_0) >= max_per_class and len(y_indices_1) >= max_per_class:
        break

print("Benign indices:", len(y_indices_0))
print("Attack indices:", len(y_indices_1))


Benign indices: 1000
Attack indices: 1000


In [ ]:
global_model.eval()

y_true, y_pred, y_prob = [], [], []

with torch.no_grad():
    for idx in y_indices_0 + y_indices_1:
        x, y = eval_dataset[idx]
        x = x.unsqueeze(0).to(DEVICE)

        prob = global_model(x).item()
        pred = int(prob > 0.3)

        y_true.append(int(y))
        y_prob.append(prob)
        y_pred.append(pred)

print("Evaluation samples used:", len(y_true))


Evaluation samples used: 2000


In [ ]:
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, roc_auc_score
)

metrics = {
    "accuracy": accuracy_score(y_true, y_pred),
    "precision": precision_score(y_true, y_pred, zero_division=0),
    "recall": recall_score(y_true, y_pred, zero_division=0),
    "f1_score": f1_score(y_true, y_pred, zero_division=0),
    "roc_auc": roc_auc_score(y_true, y_prob),
    "confusion_matrix": confusion_matrix(y_true, y_pred).tolist()
}

for k, v in metrics.items():
    print(f"{k}: {v}")


accuracy: 0.6965
precision: 0.6661031276415892
recall: 0.788
f1_score: 0.7219422812643151
roc_auc: 0.866047
confusion_matrix: [[605, 395], [212, 788]]


In [ ]:
import json

metadata = {
    "model_version": MODEL_VERSION,
    "architecture": "CNN-LSTM",
    "num_features": NUM_FEATURES,
    "sequence_length": SEQ_LEN,
    "loss_function": "BCEWithLogitsLoss",
    "pos_weight": float(POS_WEIGHT.cpu().numpy()),
    "threshold": THRESHOLD,
    "rounds": CONFIG["ROUNDS"],
    "local_epochs": CONFIG["LOCAL_EPOCHS"],
    "batch_size": CONFIG["BATCH_SIZE"]
}

meta_path = os.path.join(
    MODELS_DIR, f"model_metadata_{MODEL_VERSION}.json"
)

with open(meta_path, "w") as f:
    json.dump(metadata, f, indent=4)


In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_true, y_scores)

from sklearn.metrics import f1_score
import numpy as np

best_f1 = 0
best_thresh = 0.5

for t in np.arange(0.2, 0.8, 0.05):
    preds = (y_scores > t).astype(int)
    f1 = f1_score(y_true, preds)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(best_thresh, best_f1)


NameError: name 'y_true' is not defined

In [ ]:
import asyncio
import numpy as np
import random

NUM_FEATURES = 78

def benign_flow():
    return np.random.normal(0.05, 0.05, NUM_FEATURES)

def ddos_flow():
    x = np.random.normal(1.2, 0.8, NUM_FEATURES)
    x[:6] += 3.5
    return x

async def iot_device(device_id, queue):
    attack = False
    counter = 0

    while True:
        counter += 1
        if counter > 25:
            attack = True

        flow = ddos_flow() if attack else benign_flow()

        await queue.put({
            "device_id": device_id,
            "features": flow
        })

        await asyncio.sleep(random.uniform(0.5, 1.5))


In [ ]:
import torch
from collections import deque

SEQ_LEN = 10
THRESHOLD = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

windows = {}

async def edge_gateway(queue, model):
    while True:
        msg = await queue.get()

        device_id = msg["device_id"]
        flow = msg["features"]

        if device_id not in windows:
            windows[device_id] = deque(maxlen=SEQ_LEN)

        windows[device_id].append(flow)

        if len(windows[device_id]) < SEQ_LEN:
            continue

        sample = torch.tensor(
            np.array(windows[device_id]),
            dtype=torch.float32
        ).unsqueeze(0).to(DEVICE)

        with torch.no_grad():
            prob = torch.sigmoid(model(sample)).item()

        decision = "🚨 ATTACK" if prob > THRESHOLD else "✅ BENIGN"

        print(f"[EDGE] Device={device_id:<8} " f"Window={SEQ_LEN} " f"Prob={prob:.4f} → {decision}")



In [ ]:
  # adjust path if needed

MODEL_PATH = "/content/drive/MyDrive/FYP_FL_IDS/models/cnn_lstm_global_with_HE_25rounds_16k.pt"

model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

print("✅ Edge IDS model loaded")


✅ Edge IDS model loaded


In [ ]:
async def main():
    queue = asyncio.Queue()

    devices = [
        asyncio.create_task(iot_device("device_1", queue)),
        asyncio.create_task(iot_device("device_2", queue)),
        asyncio.create_task(iot_device("device_3", queue)),
    ]

    edge = asyncio.create_task(edge_gateway(queue, model))

    await asyncio.gather(*devices, edge)

await main()


[EDGE] Device=device_1 Window=10 Prob=0.0809 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0742 → ✅ BENIGN
[EDGE] Device=device_1 Window=10 Prob=0.0731 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0908 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0881 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0864 → ✅ BENIGN
[EDGE] Device=device_1 Window=10 Prob=0.0864 → ✅ BENIGN
[EDGE] Device=device_1 Window=10 Prob=0.0836 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0812 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0788 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0806 → ✅ BENIGN
[EDGE] Device=device_1 Window=10 Prob=0.0812 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0736 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0803 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0785 → ✅ BENIGN
[EDGE] Device=device_1 Window=10 Prob=0.0861 → ✅ BENIGN
[EDGE] Device=device_2 Window=10 Prob=0.0794 → ✅ BENIGN
[EDGE] Device=device_3 Window=10 Prob=0.0787 → ✅

CancelledError: 

In [ ]:
import torch
import numpy as np
from collections import deque

# =========================
# CONFIG
# =========================
MODEL_PATH = "models/cnn_lstm_global_with_HE_25rounds_16k.pt"
SEQ_LEN = 10
NUM_FEATURES = 78
THRESHOLD = 0.5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================
# LOAD MODEL
# =========================
import CNN_LSTM_IDS   # adjust import if needed

model = CNN_LSTM_IDS(SEQ_LEN, NUM_FEATURES).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

print("✅ Edge IDS Model Loaded")

# =========================
# TRAFFIC GENERATORS
# =========================
def benign_flow():
    return np.random.normal(0.05, 0.05, NUM_FEATURES)

def ddos_flow():
    x = np.random.normal(1.2, 0.8, NUM_FEATURES)
    x[:6] += 3.5
    return x

def slow_attack_flow():
    x = np.random.normal(0.6, 0.3, NUM_FEATURES)
    x[20:25] += 1.5
    return x

# =========================
# EDGE IDS LOGIC
# =========================
window = deque(maxlen=SEQ_LEN)

def edge_process(flow):
    window.append(flow)

    if len(window) < SEQ_LEN:
        return None

    sample = torch.tensor(np.array(window), dtype=torch.float32)
    sample = sample.unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        prob = torch.sigmoid(model(sample)).item()

    decision = "ATTACK 🚨" if prob > THRESHOLD else "BENIGN ✅"
    return prob, decision

# =========================
# DEMO RUN
# =========================
print("\n=== EDGE IDS DEMO START ===\n")

traffic_sequence = (
    ["BENIGN"] * 10 +
    ["DDoS"] * 10 +
    ["SLOW_ATTACK"] * 10
)

for t, traffic_type in enumerate(traffic_sequence):
    if traffic_type == "BENIGN":
        flow = benign_flow()
    elif traffic_type == "DDoS":
        flow = ddos_flow()
    else:
        flow = slow_attack_flow()

    result = edge_process(flow)

    if result:
        prob, decision = result
        print(f"[EDGE] Traffic={traffic_type:<12} "
              f"Prob={prob:.4f} → {decision}")

print("\n=== EDGE IDS DEMO END ===")


ModuleNotFoundError: No module named 'CNN_LSTM_IDS'